Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [52]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [53]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

In [54]:
import pandas as pd

In [55]:
%%time

transactions = pd.read_parquet('../input/hm-parquets-of-datasets/transactions_train.parquet')
customers = pd.read_parquet('../input/hm-parquets-of-datasets/customers.parquet')
articles = pd.read_parquet('../input/hm-parquets-of-datasets/articles.parquet')

CPU times: user 3.78 s, sys: 1.1 s, total: 4.88 s
Wall time: 1.55 s


In [56]:
transactions['week'].max()

104

### We have tuned with number of weeks and found 6 weeks data to give best results

In [57]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 6]

# Generating candidates

### Last purchase candidates

In [58]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: user 11 s, sys: 407 ms, total: 11.4 s
Wall time: 11.1 s


In [59]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15
104,2020-09-16,2020-09-22


In [60]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: user 368 ms, sys: 56.1 ms, total: 424 ms
Wall time: 423 ms


In [61]:
# c2weeks2shifted_weeks[28847241659200]

In [62]:
candidates_last_purchase = transactions.copy()

In [63]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: user 3.87 s, sys: 2.7 ms, total: 3.88 s
Wall time: 3.88 s


In [64]:
# candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

In [65]:
# transactions[transactions['customer_id']==272412481300040]

### Bestsellers candidates

In [66]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [67]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [68]:
# sales

In [69]:
# sales.loc[95]

In [70]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [71]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [72]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
30253369,2020-08-12,200292573348128,816563004,0.013542,2,99
30288386,2020-08-12,1037449031262554,813094004,0.013542,2,99
30237540,2020-08-12,2468546290291055,941005005,0.010831,2,99
30265271,2020-08-12,4920151714340210,817834002,0.022017,2,99
30244769,2020-08-12,5474139543167571,806388028,0.010831,2,99
...,...,...,...,...,...,...
31760188,2020-09-22,18435221511488011015,573085055,0.033881,1,104
31782234,2020-09-22,18436859303155335645,801447001,0.030492,1,104
31787251,2020-09-22,18437941771381362708,907188001,0.050831,2,104
31776022,2020-09-22,18438270306572912089,751471043,0.033881,1,104


In [73]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [74]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [75]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [76]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

# Combining transactions and candidates / negative examples

In [77]:
transactions['purchased'] = 1

In [78]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [79]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

### Add bestseller information

In [80]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [81]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [82]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [83]:
data.sort_values(['week', 'customer_id'], ignore_index=True, inplace=True)
# data.reset_index(drop=True, inplace=True)

In [84]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [85]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [86]:
train_baskets

array([13, 32, 15, ..., 14, 19, 16])

In [87]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [88]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: user 125 ms, sys: 55.6 ms, total: 180 ms
Wall time: 181 ms


# Model training

In [89]:
from lightgbm.sklearn import LGBMRanker

In [90]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    learning_rate=0.03,
    verbose=10
)

In [91]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.766338
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.159207
[LightGBM] [Debug] init for col-wise cost 0.089273 seconds, init for row-wise cost 0.291551 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.211420 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1141
[LightGBM] [Info] Number of data points in the train set: 5989162, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
CPU times: user 13.2 s, sys: 369 ms, total: 13.5 s
Wall time: 2.55 s


In [92]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9991720797336786
age 0.00023485119110721393
garment_group_no 0.00015674837467499075
article_id 0.0001330345606614913
Active 5.695676647972228e-05
postal_code 5.693596595238847e-05
department_no 4.372933462952986e-05
product_type_no 3.9739393621352696e-05
colour_group_code 3.566952896610535e-05
perceived_colour_master_id 3.0459894142758814e-05
club_member_status 3.0408131240056743e-05
section_no 9.387124845744015e-06
FN 0.0
fashion_news_frequency 0.0
index_code 0.0
perceived_colour_value_id 0.0
graphical_appearance_no 0.0
index_group_no 0.0


# Calculate predictions

In [93]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.2 µs


# Create submission

In [94]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [95]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: user 3.92 s, sys: 128 ms, total: 4.05 s
Wall time: 4.04 s


In [96]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [97]:
sub_name = 'submission'
sub.to_csv(f'{sub_name}.csv', index=False)